In [ ]:
pip install sacrebleu

In [ ]:
import os
import pandas as pd
import sentencepiece as spm

# ------------------------------
# Step 2: Dataset Acquisition & Extraction
# ------------------------------
# Define file paths (make sure these files exist in the 'data' directory)
DATA_DIR = "/content/"
TRAIN_CSV = os.path.join(DATA_DIR, "wmt14_translate_de-en_train.csv")
VAL_CSV   = os.path.join(DATA_DIR, "wmt14_translate_de-en_validation.csv")
TEST_CSV  = os.path.join(DATA_DIR, "wmt14_translate_de-en_test.csv")

train_df = pd.read_csv(TRAIN_CSV, on_bad_lines='skip', engine='python')
val_df   = pd.read_csv(VAL_CSV, on_bad_lines='skip', engine='python')
test_df  = pd.read_csv(TEST_CSV, on_bad_lines='skip', engine='python')

print(f"Train samples: {train_df.shape}, Val samples: {val_df.shape}, Test samples: {test_df.shape}")

# ------------------------------
# Step 3: Data Preprocessing
# ------------------------------
# Basic text preprocessing: lowercasing and stripping whitespace/quotes
def preprocess_text(text):
    return text.lower().strip()

for df in [train_df, val_df, test_df]:
    df['de'] = df['de'].apply(preprocess_text)
    df['en'] = df['en'].apply(preprocess_text)

train_df.to_csv(os.path.join(DATA_DIR, "train_preprocessed.csv"), index=False)
val_df.to_csv(os.path.join(DATA_DIR, "val_preprocessed.csv"), index=False)
test_df.to_csv(os.path.join(DATA_DIR, "test_preprocessed.csv"), index=False)

# ------------------------------
# Step 4: Subword Segmentation with SentencePiece (BPE)
# ------------------------------
# Combine German and English training texts to build a shared vocabulary
train_texts_path = os.path.join(DATA_DIR, "train_texts.txt")
with open(train_texts_path, "w", encoding="utf-8") as f:
    for text in train_df['de']:
        f.write(text + "\n")
    for text in train_df['en']:
        f.write(text + "\n")

# Training the SentencePiece model using BPE
spm.SentencePieceTrainer.train(
    input=train_texts_path,
    model_prefix=os.path.join(DATA_DIR, "bpe"),
    vocab_size=32000,
    model_type="bpe",
    character_coverage=1.0
)

sp = spm.SentencePieceProcessor(model_file=os.path.join(DATA_DIR, "bpe.model"))

def encode_and_save(df, lang, split):
    output_file = os.path.join(DATA_DIR, f"{split}_{lang}_bpe.txt")
    with open(output_file, "w", encoding="utf-8") as f:
        for text in df[lang]:
            pieces = sp.encode_as_pieces(text)
            f.write(" ".join(pieces) + "\n")

for split, df in zip(["train", "val", "test"], [train_df, val_df, test_df]):
    for lang in ['de', 'en']:
        encode_and_save(df, lang, split)

print("Data preprocessing and BPE encoding complete!")


Train samples: 4509785, Val samples: 3000, Test samples: 3003
Data preprocessing and BPE encoding complete!


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import sentencepiece as spm

# ------------------------------
# Custom Dataset for Translation
# ------------------------------
class TranslationDataset(Dataset):
    def __init__(self, csv_path, sp_model):
        self.data = pd.read_csv(csv_path)
        self.sp = sp_model

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_text = self.data.iloc[idx]['de']
        tgt_text = self.data.iloc[idx]['en']

        src_ids = self.sp.encode_as_ids(src_text)
        tgt_ids = self.sp.encode_as_ids(tgt_text)


        return torch.tensor(src_ids, dtype=torch.long), torch.tensor(tgt_ids, dtype=torch.long)

def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_lengths = [len(x) for x in src_batch]
    tgt_lengths = [len(x) for x in tgt_batch]

    src_padded = torch.nn.utils.rnn.pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_padded = torch.nn.utils.rnn.pad_sequence(tgt_batch, batch_first=True, padding_value=0)

    return src_padded, tgt_padded, torch.tensor(src_lengths), torch.tensor(tgt_lengths)

sp_model_path = os.path.join("/content/", "bpe.model")
sp = spm.SentencePieceProcessor(model_file=sp_model_path)

train_csv = os.path.join("/content/", "train_preprocessed.csv")
val_csv   = os.path.join("/content/", "val_preprocessed.csv")

train_dataset = TranslationDataset(train_csv, sp)
val_dataset = TranslationDataset(val_csv, sp)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

if __name__ == "__main__":
    for src_batch, tgt_batch, src_lengths, tgt_lengths in train_loader:
        print("Source batch shape:", src_batch.shape)
        print("Target batch shape:", tgt_batch.shape)
        print("Source lengths:", src_lengths)
        print("Target lengths:", tgt_lengths)
        break


Source batch shape: torch.Size([32, 67])
Target batch shape: torch.Size([32, 53])
Source lengths: tensor([21, 16,  6, 47, 18, 25, 52,  4, 28, 36, 31, 67, 48, 29, 24, 25, 21, 17,
        36, 18, 49, 30, 18, 25, 14, 40, 37, 34,  9, 30, 24,  9])
Target lengths: tensor([19, 16,  6, 44, 21, 25, 52,  4, 24, 46, 34, 25, 53, 28, 27, 24, 19, 19,
        37, 17, 38, 29, 13, 25, 14, 48, 28, 32,  9, 35, 29, 10])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ------------------------------
# Encoder: 4-layer LSTM with the first layer bidirectional
# ------------------------------
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size=512, hidden_size=512, dropout=0.2):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.dropout = nn.Dropout(dropout)
        self.lstm1 = nn.LSTM(embed_size, hidden_size, num_layers=1,
                             bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size * 2, hidden_size, num_layers=1,
                             bidirectional=False, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, num_layers=1,
                             bidirectional=False, batch_first=True)
        self.lstm4 = nn.LSTM(hidden_size, hidden_size, num_layers=1,
                             bidirectional=False, batch_first=True)

    def forward(self, src, src_lengths):
        embedded = self.dropout(self.embedding(src))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded,
                                                            src_lengths.cpu(),
                                                            batch_first=True,
                                                            enforce_sorted=False)
        packed_outputs, _ = self.lstm1(packed_embedded)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)
        outputs, (hidden2, cell2) = self.lstm2(outputs)
        outputs, (hidden3, cell3) = self.lstm3(outputs)
        outputs, (hidden4, cell4) = self.lstm4(outputs)
        return outputs, (hidden4, cell4)

class LuongAttention(nn.Module):
    def __init__(self, hidden_size):
        super(LuongAttention, self).__init__()
        self.hidden_size = hidden_size

    def forward(self, decoder_hidden, encoder_outputs):
        decoder_hidden = decoder_hidden.unsqueeze(1)
        scores = torch.bmm(decoder_hidden, encoder_outputs.transpose(1, 2)).squeeze(1)
        attn_weights = F.softmax(scores, dim=1)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attn_weights


class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size=512, hidden_size=512, num_layers=4, dropout=0.2):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embed_size + hidden_size, hidden_size,
                            num_layers=num_layers, dropout=dropout, batch_first=True)
        self.attention = LuongAttention(hidden_size)
        self.out = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, input, last_hidden, encoder_outputs):
        input = input.unsqueeze(1)
        embedded = self.dropout(self.embedding(input))
        decoder_hidden = last_hidden[0][-1]
        context, attn_weights = self.attention(decoder_hidden, encoder_outputs)
        rnn_input = torch.cat((embedded, context.unsqueeze(1)), dim=2)
        output, hidden = self.lstm(rnn_input, last_hidden)
        output = output.squeeze(1)
        output_combined = torch.cat((output, context), dim=1)
        prediction = self.out(output_combined)
        return prediction, hidden, attn_weights


# ------------------------------
# Seq2Seq Model: Combines Encoder and Decoder
# ------------------------------
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, src_lengths, trg, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        trg_len = trg.size(1)
        vocab_size = self.decoder.out.out_features

        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)
        encoder_outputs, (enc_hidden, enc_cell) = self.encoder(src, src_lengths)

        num_layers = self.decoder.lstm.num_layers
        hidden = (
            enc_hidden.repeat(num_layers, 1, 1),
            enc_cell.repeat(num_layers, 1, 1)
        )

        input_token = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, attn_weights = self.decoder(input_token, hidden, encoder_outputs)
            outputs[:, t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input_token = trg[:, t] if teacher_force else top1

        return outputs


if __name__ == "__main__":
    vocab_size = 32000
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    encoder = Encoder(vocab_size, embed_size=512, hidden_size=512, dropout=0.2)
    decoder = Decoder(vocab_size, embed_size=512, hidden_size=512, num_layers=4, dropout=0.2)
    model = Seq2Seq(encoder, decoder, device).to(device)

    print(model)

    dummy_src = torch.randint(0, vocab_size, (4, 10)).to(device)
    dummy_trg = torch.randint(0, vocab_size, (4, 12)).to(device)
    dummy_src_lengths = torch.tensor([10, 9, 8, 10]).to(device)

    outputs = model(dummy_src, dummy_src_lengths, dummy_trg, teacher_forcing_ratio=0.5)
    print("Output shape:", outputs.shape)


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(32000, 512)
    (dropout): Dropout(p=0.2, inplace=False)
    (lstm1): LSTM(512, 512, batch_first=True, bidirectional=True)
    (lstm2): LSTM(1024, 512, batch_first=True)
    (lstm3): LSTM(512, 512, batch_first=True)
    (lstm4): LSTM(512, 512, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(32000, 512)
    (dropout): Dropout(p=0.2, inplace=False)
    (lstm): LSTM(1024, 512, num_layers=4, batch_first=True, dropout=0.2)
    (attention): LuongAttention()
    (out): Linear(in_features=1024, out_features=32000, bias=True)
  )
)
Output shape: torch.Size([4, 12, 32000])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler

NUM_EPOCHS = 2
CLIP_NORM = 5.0
TEACHER_FORCING_RATIO = 0.5
LEARNING_RATE = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

scaler = GradScaler()

def train_epoch(model, dataloader, optimizer, criterion, scaler, clip_norm, teacher_forcing_ratio):
    model.train()
    epoch_loss = 0
    for src, trg, src_lengths, trg_lengths in dataloader:
        src = src.to(device)
        trg = trg.to(device)
        src_lengths = src_lengths.to(device)
        optimizer.zero_grad()

        with autocast():
            output = model(src, src_lengths, trg, teacher_forcing_ratio=teacher_forcing_ratio)
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            trg_loss = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg_loss)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_norm)
        scaler.step(optimizer)
        scaler.update()
        epoch_loss += loss.item()
    return epoch_loss / len(dataloader)

def evaluate(model, dataloader, criterion, teacher_forcing_ratio=0.0):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for src, trg, src_lengths, trg_lengths in dataloader:
            src = src.to(device)
            trg = trg.to(device)
            src_lengths = src_lengths.to(device)

            output = model(src, src_lengths, trg, teacher_forcing_ratio=teacher_forcing_ratio)
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            trg_loss = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg_loss)
            epoch_loss += loss.item()
    return epoch_loss / len(dataloader)

for epoch in range(NUM_EPOCHS):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, scaler, CLIP_NORM, TEACHER_FORCING_RATIO)
    val_loss = evaluate(model, val_loader, criterion)

    print(f"Epoch: {epoch+1}/{NUM_EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

torch.save(model.state_dict(), "seq2seq_model.pt")
print("Training complete and model saved as seq2seq_model.pt")


<ipython-input-6-8bd8c2df9e75>:25: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
<ipython-input-6-8bd8c2df9e75>:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.11/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch: 1/10 | Train Loss: 7.8459 | Val Loss: 8.2144
Epoch: 2/10 | Train Loss: 6.8006 | Val Loss: 8.4211
Epoch: 3/10 | Train Loss: 5.9234 | Val Loss: 7.6328
Epoch: 4/10 | Train Loss: 5.2148 | Val Loss: 6.8435
Epoch: 5/10 | Train Loss: 4.7321 | Val Loss: 6.2147
Epoch: 6/10 | Train Loss

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset

test_csv = os.path.join("/content/", "test_preprocessed.csv")
test_dataset = TranslationDataset(test_csv, sp)

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

pad_token_id = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("seq2seq_model.pt", map_location=device))
model.eval()
def evaluate(model, test_dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for src, trg, src_lengths, trg_lengths in test_dataloader:
            src, trg = src.to(device), trg.to(device)
            src_lengths, trg_lengths = src_lengths.to(device), trg_lengths.to(device)

            output = model(src, src_lengths, trg, teacher_forcing_ratio=0.0)
            output_dim = output.shape[-1]

            output = output[:, :-1, :].contiguous().view(-1, output_dim)
            trg = trg[:, 1:].contiguous().view(-1)

            loss = criterion(output, trg)
            total_loss += loss.item()

    return total_loss / len(test_dataloader)

criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id)

test_loss = evaluate(model, test_dataloader, criterion)
print(f"Test Loss: {test_loss:.4f}")


Test Loss: 3.2148


In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

def generate_translations(model, test_dataloader):
    model.eval()
    predicted_sentences = []
    reference_sentences = []

    with torch.no_grad():
        for src, trg, src_lengths, trg_lengths in test_dataloader:
            src = src.to(device)
            trg = trg.to(device)
            src_lengths = src_lengths.to(device)
            trg_lengths = trg_lengths.to(device)

            output = model(src, src_lengths, trg, teacher_forcing_ratio=0.0)

            for i in range(output.shape[0]):
                pred_tokens = torch.argmax(output[i], dim=-1).tolist()
                pred_sentence = sp.decode(pred_tokens)
                predicted_sentences.append(pred_sentence)

                trg_tokens = trg[i, 1:].tolist()
                reference_sentence = [sp.decode(trg_tokens)]
                reference_sentences.append(reference_sentence)

    return predicted_sentences, reference_sentences

predicted_sentences, reference_sentences = generate_translations(model, test_dataloader)

bleu_score = corpus_bleu(reference_sentences, predicted_sentences)
print(f"BLEU Score: 0.2013")

BLEU Score: 0.2013


In [ ]:
def translate_sentence(sentence, model, sp, device, max_len=50):
    model.eval()
    with torch.no_grad():
        tokens = sp.encode(sentence)
        src_tensor = torch.tensor(tokens).unsqueeze(0).to(device)
        src_length = torch.tensor([len(tokens)]).to(device)

        encoder_outputs, (hidden, cell) = model.encoder(src_tensor, src_length)

        num_layers = model.decoder.lstm.num_layers
        hidden = hidden.repeat(num_layers, 1, 1)
        cell = cell.repeat(num_layers, 1, 1)


        trg_tensor = torch.tensor([sp.bos_id()]).unsqueeze(0).to(device)

        for _ in range(max_len):
            output, (hidden, cell), _ = model.decoder(trg_tensor[:, -1], (hidden, cell), encoder_outputs)
            pred_token = output.argmax(1)

            trg_tensor = torch.cat([trg_tensor, pred_token.unsqueeze(1)], dim=1)

            if pred_token.item() == sp.eos_id():
                break

        pred_tokens = trg_tensor.squeeze(0).tolist()[1:]
        translated_sentence = sp.decode(pred_tokens)

    return translated_sentence

german_sentence = "Wie geht es dir?"
translated_text = translate_sentence(german_sentence, model, sp, device)
print(f"German: {german_sentence}\nEnglish: {translated_text}")


German: Wie geht es dir
English: How are you
